<a href="https://colab.research.google.com/github/arimbawa/Big-Data/blob/main/04_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
url = "https://news.detik.com/berita"

response = requests.get(url)
print(response.status_code)  # 200 berarti sukses

200


In [8]:
soup = BeautifulSoup(response.text, "html.parser")

# Cari semua judul berita (tag <h3>)
berita = soup.find_all("h3", class_="media__title")

for b in berita:
    judul = b.get_text(strip=True)
    link = b.find("a")["href"]
    print(judul, "=>", link)


22 September Memperingati Hari Apa? Ini Penjelasannya => https://news.detik.com/berita/d-8123818/22-september-memperingati-hari-apa-ini-penjelasannya
Sejumlah Proyek Galian di Jakarta Bikin Macet, Perjalanan Bus TransJ Terlambat => https://news.detik.com/berita/d-8123796/sejumlah-proyek-galian-di-jakarta-bikin-macet-perjalanan-bus-transj-terlambat
Legislator Harap Prabowo Suarakan Kedaulatan Palestina di Sidang Umum PBB => https://news.detik.com/berita/d-8123795/legislator-harap-prabowo-suarakan-kedaulatan-palestina-di-sidang-umum-pbb
Curi Motor Mahasiswi di Bogor, 'Partner in Crime' Diringkus Warga => https://news.detik.com/berita/d-8123763/curi-motor-mahasiswi-di-bogor-partner-in-crime-diringkus-warga
Ada Proyek Galian, Lalin di Cawang Arah Otista Macet => https://news.detik.com/berita/d-8123754/ada-proyek-galian-lalin-di-cawang-arah-otista-macet
9 ABK WNI Terjebak Hampir Setahun dalam Kapal di Mozambik Berhasil Dievakuasi => https://news.detik.com/berita/d-8123746/9-abk-wni-terjebak

In [9]:
import csv

with open("berita_detik.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Judul", "Link"])

    for b in berita:
        judul = b.get_text(strip=True)
        link = b.find("a")["href"]
        writer.writerow([judul, link])


In [10]:
import csv
import requests
from bs4 import BeautifulSoup
import time

# Buka file CSV yang sudah ada
with open("berita_detik.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)  # baca CSV sebagai dictionary
    articles = list(reader)     # simpan semua data ke list

# Loop setiap baris (judul + link)
for row in articles:
    judul = row["Judul"]
    link = row["Link"]
    print(f"\n=== {judul} ===")
    print(f"Link: {link}")

    try:
        # Ambil halaman artikel
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            # Cari isi artikel (Detik biasanya dalam <div class="detail__body-text">)
            isi = soup.find("div", class_="detail__body-text")

            if isi:
                paragraphs = [p.get_text(strip=True) for p in isi.find_all("p")]
                artikel = "\n".join(paragraphs)
                print(artikel[:500], "...")  # tampilkan 500 karakter pertama
            else:
                print("Isi artikel tidak ditemukan.")
        else:
            print("Gagal ambil halaman:", response.status_code)
    except Exception as e:
        print("Error:", e)

    time.sleep(2)  # kasih jeda biar tidak kena blokir server


=== 22 September Memperingati Hari Apa? Ini Penjelasannya ===
Link: https://news.detik.com/berita/d-8123818/22-september-memperingati-hari-apa-ini-penjelasannya
Setiap tanggal 22 September, dunia memperingatiHari Bebas Kendaraan Bermotor SeduniaatauWorld Car-Free Day. Momentum ini mengajak masyarakat untuk meninggalkan kendaraan pribadi sejenak dan merasakan manfaat ruang kota tanpa polusi serta kebisingan lalu lintas.
Peringatan ini tidak hanya simbolik, tetapi juga menjadi ajakan nyata untuk beralih pada transportasi ramah lingkungan seperti berjalan kaki dan bersepeda. Lantas, bagaimana sejarah, tujuan, dan cara merayakan Hari Bebas Kendaraan Bermoto ...

=== Sejumlah Proyek Galian di Jakarta Bikin Macet, Perjalanan Bus TransJ Terlambat ===
Link: https://news.detik.com/berita/d-8123796/sejumlah-proyek-galian-di-jakarta-bikin-macet-perjalanan-bus-transj-terlambat
Lalu lintas di wilayah Cawang arah Otista Jakarta Timur macet akibat adanya proyek galian pagi ini. Akibatnya bus Transja

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import time

# Buka file CSV lama yang berisi Judul & Link
with open("berita_detik.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    articles = list(reader)

# Buka file CSV baru untuk menyimpan hasil lengkap
with open("berita_lengkap.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Judul", "Link", "Isi"])  # header kolom

    # Loop setiap artikel
    for row in articles:
        judul = row["Judul"]
        link = row["Link"]
        print(f"\nMengambil artikel: {judul}")

        try:
            response = requests.get(link)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")

                # Cari isi artikel (untuk Detik: div.detail__body-text)
                isi = soup.find("div", class_="detail__body-text")

                if isi:
                    paragraphs = [p.get_text(strip=True) for p in isi.find_all("p")]
                    artikel = " ".join(paragraphs)
                else:
                    artikel = "Isi artikel tidak ditemukan."

                # Tulis ke file CSV baru
                writer.writerow([judul, link, artikel])
                print("Artikel berhasil disimpan.")
            else:
                print("Gagal ambil halaman:", response.status_code)
        except Exception as e:
            print("Error:", e)

        time.sleep(2)  # jeda 2 detik biar aman



Mengambil artikel: Ada Proyek Galian, Lalin di Cawang Arah Otista Macet
Artikel berhasil disimpan.

Mengambil artikel: 9 ABK WNI Terjebak Hampir Setahun dalam Kapal di Mozambik Berhasil Dievakuasi
Artikel berhasil disimpan.

Mengambil artikel: Istana hingga DPR Merespons Sorotan Sirine-Rotator
Artikel berhasil disimpan.

Mengambil artikel: Begini Langkah TransJakarta Usai 3 Kali Kecelakaan Bus dalam Sebulan
Artikel berhasil disimpan.

Mengambil artikel: Mengenal Sufi, Sosok Jaksa Pembela Perempuan & Anak-anak di Tanah Papua
Artikel berhasil disimpan.

Mengambil artikel: Polisi Surati Dishub, Tambah Rambu-rambu Imbas Kecelakaan Flyover Pancoran
Artikel berhasil disimpan.

Mengambil artikel: Lewat Pemberdayaan BUMDes, Kejaksaan Dorong Kemandirian Warga Kepahiang
Artikel berhasil disimpan.

Mengambil artikel: Kapolri Bentuk Tim Transformasi Reformasi Polri
Artikel berhasil disimpan.

Mengambil artikel: Flyover Pancoran Rawan Kecelakaan, Anggota DPRD DKI Soroti Pemeliharaan Jalan
Artikel 